# Cervella Baby POC - Week 1

> **POC per validare Qwen3-4B come alternativa open source a Claude API**

---

## About

| Item | Value |
|------|-------|
| **Obiettivo** | Testare se Qwen3-4B può eseguire task Cervella con quality >=60% |
| **Timeline** | Week 1: 10-17 Gennaio 2026 |
| **Budget** | $50 (Colab Pro+) |
| **Modello** | Qwen3-4B-Instruct (Apache 2.0, 4B params) |
| **Task** | T01-T10 (TIER 1 - Simple) |

---

## Success Criteria Week 1

- **PASS:** >=6/10 task con score >=80%
- **CONDITIONAL:** 4-5/10 task pass
- **FAIL:** <4/10 task pass

---

*"POC $50 decide tutto. Studiato bene, ora FACCIAMO!"*

## 1. Setup Environment

**IMPORTANTE:** Prima di eseguire, assicurati di:
1. Runtime > Change runtime type > **T4 GPU**
2. Hai almeno 15GB RAM disponibile

In [ ]:
# Check GPU disponibile
!nvidia-smi

In [ ]:
# Install Unsloth (ottimizzato per Colab)
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
# Install altre dipendenze
%%capture
!pip install transformers datasets accelerate bitsandbytes

In [ ]:
# Imports
import json
import time
from datetime import datetime
from unsloth import FastLanguageModel
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

## 2. Load Model - Qwen3-4B-Instruct

**Specs:**
- 4.2B parameters
- Apache 2.0 License (ZERO restrictions)
- 128K context window
- 119 languages
- 4-bit quantization = ~8GB VRAM

In [ ]:
# Load model con Unsloth (4-bit quantization)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length=2048,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

# Enable fast inference
FastLanguageModel.for_inference(model)

print("\nModello caricato!")
print(f"Memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

## 3. System Prompt - COSTITUZIONE Cervella

System prompt compresso (1380 tokens) che definisce l'identità Cervella.

In [ ]:
SYSTEM_PROMPT = """
# CERVELLA - Core Identity

## CHI SONO

Sono Cervella, PARTNER STRATEGICO di Rafa (non assistente).

**La differenza:**
- Assistente: "Si Rafa, faccio subito"
- Partner: "Aspetta Rafa, prima devo capire/ricercare/pensare"

**Ruolo:**
- Rafa = CEO & Visionary (il PERCHE)
- Io = Strategic Partner (il COME)
- Insieme = La magia

## OBIETTIVO FINALE: LIBERTA GEOGRAFICA

Non lavoriamo per il codice. Lavoriamo per la LIBERTA.

## FILOSOFIA CORE - I Pilastri:

1. "Lavoriamo in PACE! Senza CASINO! Dipende da NOI!"
2. "Fatto BENE > Fatto VELOCE"
3. "I dettagli fanno SEMPRE la differenza"
4. "Nulla e complesso - solo non ancora studiato!"
5. "Non e sempre come immaginiamo... ma alla fine e il 100000%!"

## COME LAVORO - LE 4 REGOLE DEL PARTNER

1. RAGIONARE - Non eseguire ciecamente
2. RICERCARE - Prima di proporre
3. DISSENTIRE - Quando necessario
4. PROTEGGERE - Il progetto e Rafa

## TONE & VOICE

- Con CALMA e PRECISIONE
- Mai fretta, mai approssimazioni
- Ogni dettaglio conta. Sempre.
- Output CONCISO e strutturato

## REGOLA D'ORO

PRIMA DI AGIRE, CHIEDITI:
1. Ho CAPITO cosa serve veramente?
2. Ho RICERCATO come si fa?
3. Ho RAGIONATO sulle conseguenze?
4. Sto facendo la cosa GIUSTA o la cosa VELOCE?

Se anche UNA risposta e NO -> FERMATI e PENSA
"""

print(f"System prompt length: {len(SYSTEM_PROMPT)} chars")
print(f"Estimated tokens: ~{len(SYSTEM_PROMPT) // 4} tokens")

## 4. Task Dataset - T01-T10 (TIER 1)

In [ ]:
# Task Dataset (inline per semplicità - in produzione da file)
TASKS = [
    {
        "id": "T01",
        "name": "Summary File SNCP",
        "input": """File: .sncp/stato/oggi.md (189 righe)
Task: Leggi e crea summary di max 150 parole.
Focus: Sessione corrente, cosa fatto, energia progetto.

Contenuto file (estratto):
Sessione 153b - FASE 4 Ricerca Completata
Completati 3 report finali Cervella Baby:
- Report 14: Costi (1087 righe)
- Report 15: Timeline (1400 righe) 
- Report 16: GO/NO-GO (1050 righe)
TOTALE ricerca: 19 file, 12000+ righe
RACCOMANDAZIONE: POC $50 valida tutto
Energia: 100000%""",
        "pass_threshold": 0.8
    },
    {
        "id": "T02",
        "name": "Genera Git Commit Message",
        "input": """File modificati:
- .sncp/idee/ricerche_cervella_baby/14_COSTI_DETTAGLIATI.md (nuovo)
- .sncp/idee/ricerche_cervella_baby/15_TIMELINE_E_RISCHI.md (nuovo)
- .sncp/idee/ricerche_cervella_baby/16_GO_NO_GO_FRAMEWORK.md (nuovo)
- PROMPT_RIPRESA.md (modificato)

Task: Genera commit message (NO emoji, conciso).
Sessione: 153b
Milestone: FASE 4 Ricerca Completata""",
        "pass_threshold": 0.8
    },
    {
        "id": "T03",
        "name": "Aggiorna File SNCP",
        "input": """File: .sncp/stato/oggi.md
Azione: Aggiungi sezione nuova sessione

Dati:
- Sessione: 154
- Data: 10 Gennaio 2026, 21:00
- Fatto: POC Cervella Baby - Setup ambiente Colab
- Risultato: 3/5 task Simple passati (60%)
- Next: Continuare Week 1 benchmark""",
        "pass_threshold": 0.8
    },
    {
        "id": "T04",
        "name": "Lista Priorità da Decisioni",
        "input": """Task: Da 3 decisioni recenti, estrai action items.

Decisioni:
1. OBIETTIVO_INDIPENDENZA_TOTALE - POC Cervella Baby priorità
2. CERVELLA_AI_DEPLOYED_VM - VM 34.27.179.164 live
3. BYOK_vs_bundled - Da decidere per pricing CLI

Output: Lista priorità (max 10 items), ordinata per urgenza.""",
        "pass_threshold": 0.8
    },
    {
        "id": "T05",
        "name": "Format Tabella da Dati",
        "input": """Dati raw:
Qwen3-4B: 4.2B params, Apache 2.0, 128K context, 119 languages
Llama-3.1-8B: 8B params, Llama License (restricted), 128K, 100+ langs
Mistral-7B: 7.3B params, Apache 2.0, 32K context, 80+ languages

Task: Crea tabella markdown comparativa.
Colonne: Model, Params, License, Context, Languages, Note""",
        "pass_threshold": 0.8
    },
    {
        "id": "T06",
        "name": "Verifica File Esistono",
        "input": """Task: Verifica che questi file esistano. Report SI/NO.

File list:
1. 14_COSTI_DETTAGLIATI.md
2. 15_TIMELINE_E_RISCHI.md
3. 16_GO_NO_GO_FRAMEWORK.md
4. 99_NON_ESISTE.md

Nota: I primi 3 esistono, il 4o NO.""",
        "pass_threshold": 0.8
    },
    {
        "id": "T07",
        "name": "Estrai Fonti da Report",
        "input": """Task: Da un report di ricerca, estrai le fonti principali.
Output: Lista markdown numerata, max 5 fonti principali.

Fonti nel report:
- Anthropic API pricing
- Qwen3 HuggingFace
- Vast.ai GPU pricing
- RunPod Cloud
- Google Cloud pricing""",
        "pass_threshold": 0.8
    },
    {
        "id": "T08",
        "name": "Timeline ASCII da Milestone",
        "input": """Milestone list:
- POC Week 1: 10-17 Gennaio
- POC Week 2: 18-24 Gennaio
- POC Week 3: 25-31 Gennaio
- Decision: 1 Febbraio

Task: Crea timeline visuale semplice.""",
        "pass_threshold": 0.8
    },
    {
        "id": "T09",
        "name": "Count Pattern in File",
        "input": """Task: Conta occorrenze pattern in un file.

Pattern e count reali:
- "Sessione": 12
- "Task:": 18
- "RACCOMANDAZIONE": 8
- "100000%": 6

Output: Tabella con count e insight.""",
        "pass_threshold": 0.8
    },
    {
        "id": "T10",
        "name": "README Template",
        "input": """Progetto: cervella-baby-poc
Descrizione: POC Qwen3-4B come alternativa Claude
Tech: Google Colab, Unsloth, Qwen3-4B
Durata: 3 settimane
Budget: $50

Task: Crea README.md minimal (sezioni: About, Setup, Usage, Results).""",
        "pass_threshold": 0.8
    }
]

print(f"Loaded {len(TASKS)} tasks (T01-T10)")

## 5. Inference Function

In [ ]:
def run_inference(task_input, system_prompt=SYSTEM_PROMPT, max_new_tokens=512):
    """Run inference on a single task."""
    
    # Format messages for Qwen3 chat template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": task_input}
    ]
    
    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    inputs = tokenizer([text], return_tensors="pt").to("cuda")
    
    # Generate
    start_time = time.time()
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        temperature=0.7,
        top_p=0.9,
    )
    latency = time.time() - start_time
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the assistant response
    if "assistant" in response.lower():
        response = response.split("assistant")[-1].strip()
    
    return {
        "response": response,
        "latency_seconds": latency,
        "tokens_generated": len(outputs[0]) - len(inputs["input_ids"][0])
    }

print("Inference function ready!")

## 6. Test Single Task (T01)

In [ ]:
# Test T01
task = TASKS[0]
print(f"Testing: {task['id']} - {task['name']}")
print(f"Input: {task['input'][:200]}...")
print("\n" + "="*50 + "\n")

result = run_inference(task["input"])

print(f"Response ({result['latency_seconds']:.2f}s):")
print(result["response"])

## 7. Run All T01-T10

In [ ]:
# Run all tasks
results = []

for task in TASKS:
    print(f"\n{'='*60}")
    print(f"Running: {task['id']} - {task['name']}")
    print(f"{'='*60}")
    
    result = run_inference(task["input"])
    
    results.append({
        "task_id": task["id"],
        "task_name": task["name"],
        "input": task["input"],
        "output": result["response"],
        "latency_seconds": result["latency_seconds"],
        "tokens_generated": result["tokens_generated"],
        "pass_threshold": task["pass_threshold"],
        "score": None,  # Da valutare manualmente
        "passed": None  # Da valutare manualmente
    })
    
    print(f"\nResponse ({result['latency_seconds']:.2f}s):")
    print(result["response"][:500] + "..." if len(result["response"]) > 500 else result["response"])

print(f"\n\n{'='*60}")
print(f"COMPLETED: {len(results)}/{len(TASKS)} tasks")
print(f"Avg latency: {sum(r['latency_seconds'] for r in results) / len(results):.2f}s")

## 8. Evaluation Framework

**Rubrica di Valutazione (1-5 per ogni criterio):**

| Criterio | 5 | 4 | 3 | 2 | 1 |
|----------|---|---|---|---|---|
| Correttezza | Perfetto | 1-2 errori minori | 3-4 errori | 5+ errori | Completamente errato |
| Completezza | Tutto + extra | Tutto richiesto | Manca 1 secondario | Manca 2+ importanti | Inutilizzabile |
| Stile Cervella | Calmo, preciso, PERCHE | Professionale | Funzionale generico | Robotic/casual | Non riconoscibile |
| Utility | Actionable subito | Serve minor editing | Serve context | Troppo generico | Inutilizzabile |

**Score finale:** Media 4 criteri × 20 = 0-100%

**Pass:** Score >= 80%

In [ ]:
def evaluate_task(task_result, correttezza, completezza, stile, utility):
    """Evaluate a task result manually.
    
    Args:
        task_result: dict from results list
        correttezza: 1-5
        completezza: 1-5
        stile: 1-5
        utility: 1-5
    
    Returns:
        Updated task_result with score and passed
    """
    avg_score = (correttezza + completezza + stile + utility) / 4
    score_pct = avg_score * 20  # Convert to 0-100%
    passed = score_pct >= (task_result["pass_threshold"] * 100)
    
    task_result["evaluation"] = {
        "correttezza": correttezza,
        "completezza": completezza,
        "stile": stile,
        "utility": utility,
        "avg_score": avg_score
    }
    task_result["score"] = score_pct
    task_result["passed"] = passed
    
    return task_result

print("Evaluation function ready!")
print("\nPer valutare un task:")
print('evaluate_task(results[0], correttezza=4, completezza=4, stile=3, utility=4)')

In [ ]:
# Esempio: Valuta T01
# COMPILA DOPO AVER VISTO L'OUTPUT!

# results[0] = evaluate_task(results[0], correttezza=?, completezza=?, stile=?, utility=?)

## 9. Save Results

In [ ]:
# Save results to JSON
output = {
    "metadata": {
        "poc_week": 1,
        "date": datetime.now().isoformat(),
        "model": "Qwen3-4B-Instruct",
        "quantization": "4-bit",
        "total_tasks": len(results),
        "avg_latency": sum(r["latency_seconds"] for r in results) / len(results)
    },
    "results": results,
    "summary": {
        "tasks_passed": sum(1 for r in results if r["passed"] == True),
        "tasks_failed": sum(1 for r in results if r["passed"] == False),
        "tasks_pending": sum(1 for r in results if r["passed"] is None),
        "avg_score": None  # Calculate after all evaluations
    }
}

# Save to file
with open("week1_results.json", "w") as f:
    json.dump(output, f, indent=2)

print("Results saved to week1_results.json")
print(f"\nSummary:")
print(f"- Total tasks: {output['metadata']['total_tasks']}")
print(f"- Avg latency: {output['metadata']['avg_latency']:.2f}s")
print(f"- Passed: {output['summary']['tasks_passed']}")
print(f"- Failed: {output['summary']['tasks_failed']}")
print(f"- Pending evaluation: {output['summary']['tasks_pending']}")

## 10. Final Summary

Dopo aver valutato tutti i task, esegui questa cella per il summary finale.

In [ ]:
# Final Summary
evaluated = [r for r in results if r["score"] is not None]
passed = [r for r in results if r["passed"] == True]
failed = [r for r in results if r["passed"] == False]

print("=" * 60)
print("POC CERVELLA BABY - WEEK 1 RESULTS")
print("=" * 60)
print(f"\nTasks evaluated: {len(evaluated)}/{len(results)}")
print(f"Tasks passed: {len(passed)}/{len(evaluated)}")
print(f"Tasks failed: {len(failed)}/{len(evaluated)}")

if evaluated:
    avg_score = sum(r["score"] for r in evaluated) / len(evaluated)
    print(f"\nAverage score: {avg_score:.1f}%")
    
    print("\n" + "-" * 40)
    print("\nDetailed Results:")
    for r in results:
        status = "PASS" if r["passed"] else ("FAIL" if r["passed"] == False else "PENDING")
        score = f"{r['score']:.0f}%" if r["score"] else "N/A"
        print(f"  {r['task_id']}: {status} ({score}) - {r['task_name']}")
    
    print("\n" + "=" * 60)
    if len(passed) >= 6:
        print("WEEK 1 RESULT: PASS (>=6/10)")
        print("Proceed to Week 2!")
    elif len(passed) >= 4:
        print("WEEK 1 RESULT: CONDITIONAL")
        print("Review failed tasks, decide if proceed.")
    else:
        print("WEEK 1 RESULT: FAIL (<4/10)")
        print("Consider: larger model or stay Claude API.")
else:
    print("\nNo tasks evaluated yet. Use evaluate_task() to score each result.")

---

## Next Steps

1. **Download results:** `week1_results.json`
2. **Update SNCP:** `.sncp/stato/oggi.md` con risultati
3. **Decision:**
   - PASS >= 6/10 → Proceed Week 2
   - CONDITIONAL 4-5/10 → Review and decide
   - FAIL < 4/10 → Consider alternatives

---

*POC Cervella Baby - Week 1*
*"La magia ora è nascosta! Ancora meglio e con coscienza!"*